# `2017 - 2022` Analysis

In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# Notes

> Code does a good job of filtering for `2017 - 2018`. 

> Needs to be tuned up for `2019 - 2022` because file changes occurred again, but mainly just file location

> For the sake of time, at the moment will settle for just moving on to include only directly relevant data to our analysis because of complication making the dataset play nice for `2017-2018` and `2019-2022`.

> For `2019 - 2022` need to find `DEATHINED`. Also for all years need dictionary to translate `SEX`, `EDDISCHARGEDISPOSITION`, `DEATHINED`, `HOSPDISCHARGEDISPOSITION`.

In [4]:
data_fp = '/Users/JakeCanfield/Documents/Trauma_Surgery_Research/Data/Raw_data/PUF AY %i/CSV/'
years = range(2017, 2023)
thoracotomy_codes = ['0WJB0ZZ', '0WJ90ZZ', '02JA0ZZ', '02JY0ZZ', '0BJL0ZZ', '0BJK0ZZ', '0BJQ0ZZ','0WJ80ZZ', '0WJC0ZZ', '0WJD0ZZ', '02VW0CZ', '02QA0ZZ', '3E080GC']
sternotomy_codes = ['0P800ZZ']

mechanism_code_dict = {1:'Cut/pierce', 2:'Drowning/submersion', 3:'Fall', 4:'Fire/flame', 5:'Hot object/substance', 6:'Firearm', 7:'Machinery', 8:'MVT Occupant', 9:'MVT Motorcyclist', 10:'MVT Pedal cyclist', 11:'MVT Pedestrian', 12:'MVT Unspecified', 13:'MVT Other', 14:'Pedal cyclist, other', 15:'Pedestrian, other', 16:'Transport, other', 17:'Natural/environmental,  Bites and stings', 18:'Natural/environmental,  Other', 19:'Overexertion', 20:'Poisoning', 21:'Struck by, against', 22:'Suffocation', 23:'Other specified and classifiable', 24:'Other specified, not elsewhere classifiable', 25:'Unspecified', 26:'Adverse effects, medical care', 27:'Adverse effects, drugs'} # As noted in PUF dictionary
trauma_type_code_dict = {1:'Blunt', 2:'Penetrating', 3:'Burn', 4:'Other/unspecified', 9:'Activity Code - Not Valid as a Primary E-Code'} # As noted in PUF Dictionary
sex_code_dict = {1:'Male', 2:'Female'}
eddischarge_code_dict = {1: 'Floor bed (general admission, non-specialty unit bed)', 2: 'Observation unit (unit that provides < 24 hour stays)', 3: 'Telemetry/step-down unit (less acuity than ICU)', 4: 'Home with services', 5: 'Deceased/expired', 6: 'Other (jail, institutional care, mental health, etc.)', 7: 'Operating Room', 8: 'Intensive Care Unit (ICU)', 9: 'Home without services', 10: 'Left against medical advice', 11: 'Transferred to another hospital'}
hospdischarge_disposition_code_dict = {1: 'Discharged/Transferred to a short-term general hospital for inpatient care', 2: 'Discharged/Transferred to an Intermediate Care Facility (ICF)', 3: 'Discharged/Transferred to home under care of organized home health service', 4: 'Left against medical advice or discontinued care', 5: 'Deceased/Expired', 6: 'Discharged to home or self-care (routine discharge)', 7: 'Discharged/Transferred to Skilled Nursing Facility (SNF)', 8: 'Discharged/Transferred to hospice care', 10: 'Discharged/Transferred to court/law enforcement', 11: 'Discharged/Transferred to inpatient rehab or designated unit', 12: 'Discharged/Transferred to Long Term Care Hospital (LTCH)', 13: 'Discharged/Transferred to a psychiatric hospital or psychiatric distinct part unit of a hospital', 14: 'Discharged/Transferred to another type of institution not defined elsewhere'}


for year in years:
    
    if year in range(2017, 2019):
        TRAUMA_df = pd.read_csv(data_fp%year + 'PUF_TRAUMA.csv')
        ICDPROCEDURE_df = pd.read_csv(data_fp%year + 'PUF_ICDPROCEDURE.csv')
        ICDPROCEDURE_LOOKUP_df = pd.read_csv(data_fp%year + 'PUF_ICDPROCEDURE_LOOKUP.csv')
        ECODE_LOOKUP_df = pd.read_csv(data_fp%year + 'PUF_ECODE_LOOKUP.csv')
    elif year in range(2019, 2023):
        TRAUMA_df = pd.read_csv(data_fp%year + 'PUF_TRAUMA.csv')
        ICDPROCEDURE_df = pd.read_csv(data_fp%year + 'PUF_ICDPROCEDURE.csv')
        ICDPROCEDURE_LOOKUP_df = pd.read_csv(data_fp%year + 'PUF_ICDPROCEDURE_LOOKUP.csv')
        ECODE_LOOKUP_df = pd.read_csv(data_fp%year + 'PUF_ECODE_LOOKUP.csv')
    
    TRAUMA_df.columns = map(str.upper, TRAUMA_df.columns)
    ICDPROCEDURE_df.columns = map(str.upper, ICDPROCEDURE_df.columns)
    ICDPROCEDURE_LOOKUP_df.columns = map(str.upper, ICDPROCEDURE_LOOKUP_df.columns)
    ECODE_LOOKUP_df.columns = map(str.upper, ECODE_LOOKUP_df.columns)
    
    if year in range(2019, 2023):
        ICDPROCEDURE_df['PROCEDUREMINS'] = ICDPROCEDURE_df['HOSPITALPROCEDURESTARTHRS']*60
        ICDPROCEDURE_df['PROCEDUREDAYS'] = ICDPROCEDURE_df['HOSPITALPROCEDURESTARTDAYS']
    
    ICDPROCEDURE_df = ICDPROCEDURE_df.loc[:, ['INC_KEY', 'ICDPROCEDURECODE', 'PROCEDUREMINS', 'PROCEDUREDAYS']]
    potential_thoracotomy = ICDPROCEDURE_df.loc[ICDPROCEDURE_df['ICDPROCEDURECODE'].isin(thoracotomy_codes)]
    potential_thoracotomy = potential_thoracotomy.loc[potential_thoracotomy['PROCEDUREMINS'] <= 20.0]
    potential_thoracotomy = potential_thoracotomy.loc[potential_thoracotomy['PROCEDUREDAYS'] <= 1.0]
    potential_thoracotomy_cases = np.unique(potential_thoracotomy['INC_KEY'])
    sternotomy_cases = ICDPROCEDURE_df.loc[ICDPROCEDURE_df['ICDPROCEDURECODE'].isin(sternotomy_codes)]
    sternotomy_cases = np.unique(sternotomy_cases['INC_KEY'])
    thoracotomy_cases = [x for x in potential_thoracotomy_cases if x not in sternotomy_cases]
    ICDPROCEDURE_df = ICDPROCEDURE_df.loc[ICDPROCEDURE_df['INC_KEY'].isin(thoracotomy_cases)]
    
    ICDPROCEDURE_LOOKUP_df = ICDPROCEDURE_LOOKUP_df.loc[:, ['ICDPROCEDURECODE', 'ICDPROCEDURECODE_DESC']]
    ECODE_LOOKUP_df = ECODE_LOOKUP_df.loc[:, ['ECODE', 'ECODE_DESC', 'MECHANISM', 'TRAUMATYPE']]
    
    procedure_dict = dict(zip(ICDPROCEDURE_LOOKUP_df['ICDPROCEDURECODE'], ICDPROCEDURE_LOOKUP_df['ICDPROCEDURECODE_DESC'])) # create dictionary
    ICDPROCEDURE_df['ICDPROCEDURECODE'] = ICDPROCEDURE_df['ICDPROCEDURECODE'].replace(procedure_dict) # implement dictionary
    
    trauma_cols = ['INC_KEY', 'PRIMARYECODEICD10', 'AGEYEARS', 'SEX', 'EDDISCHARGEDISPOSITION', 'HOSPDISCHARGEDISPOSITION']
    #trauma_groups = ['EMS', 'BLOOD', 'PLASMA', 'PLATELETS', 'CRYOPRECIPITATE', 'HMRRHGCTRLSURG', 'WITHDRAWALLST']
    #for group in trauma_groups:
        #trauma_cols.extend([col for col in TRAUMA_df if col.startswith(group)])
    #trauma_cols = [x for x in trauma_cols if 'BIU' not in x]
    TRAUMA_df = TRAUMA_df.loc[:, trauma_cols]
    TRAUMA_df = TRAUMA_df.loc[TRAUMA_df['INC_KEY'].isin(np.unique(ICDPROCEDURE_df['INC_KEY']))]
    
    ecode_dict = dict(zip(ECODE_LOOKUP_df['ECODE'], ECODE_LOOKUP_df['ECODE_DESC'])) # create dictionary
    TRAUMA_df['PRIMARYECODEICD10'] = TRAUMA_df['PRIMARYECODEICD10'].replace(ecode_dict) # implement dictionary
    
    mechanism_dict = dict(zip(ECODE_LOOKUP_df['ECODE_DESC'], ECODE_LOOKUP_df['MECHANISM'])) # create dictionary
    TRAUMA_df['MECHANISM'] = TRAUMA_df['PRIMARYECODEICD10'].map(mechanism_dict) # implement dictionary
    
    traumatype_dict = dict(zip(ECODE_LOOKUP_df['ECODE_DESC'], ECODE_LOOKUP_df['TRAUMATYPE'])) # create dictionary
    TRAUMA_df['TRAUMATYPE'] = TRAUMA_df['PRIMARYECODEICD10'].map(traumatype_dict) # implement dictionary
    
    TRAUMA_df['MECHANISM'] = TRAUMA_df['MECHANISM'].replace(mechanism_code_dict) # implement dictionary
    
    TRAUMA_df['TRAUMATYPE'] = TRAUMA_df['TRAUMATYPE'].replace(trauma_type_code_dict) # implement dictionary
    
    TRAUMA_df['SEX'] = TRAUMA_df['SEX'].replace(sex_code_dict)
    
    TRAUMA_df['EDDISCHARGEDISPOSITION'] = TRAUMA_df['EDDISCHARGEDISPOSITION'].replace(eddischarge_code_dict) # implement dictionary
    
    TRAUMA_df['HOSPDISCHARGEDISPOSITION'] = TRAUMA_df['HOSPDISCHARGEDISPOSITION'].replace(hospdischarge_disposition_code_dict) # implement dictionary
    
    TRAUMA_df.to_csv('/Users/JakeCanfield/Documents/Trauma_Surgery_Research/data/Combined_data/TRAUMA_df_%i.csv'%year)
    ICDPROCEDURE_df.to_csv('/Users/JakeCanfield/Documents/Trauma_Surgery_Research/data/Combined_data/ICDPROCEDURE_df_%i.csv'%year)
    
    display(ICDPROCEDURE_df.head())
    display(TRAUMA_df.head())

(997970, 330)

(6095201, 6)

(80635, 3)

(3053, 6)

(6095201, 4)

...

(2063, 4)

(1232956, 6)

(2864, 6)

,INC_KEY,ICDPROCEDURECODE,PROCEDUREMINS,PROCEDUREDAYS
4686,220063640246,"Insertion of Endotracheal Airway into Trachea,...",3.0,1.0
4687,220063640246,Drainage of Right Pleural Cavity with Drainage...,4.2,1.0
4688,220063640246,Drainage of Left Pleural Cavity with Drainage ...,4.8,1.0
4689,220063640246,"Inspection of Right Pleural Cavity, Open Approach",4.8,1.0
4690,220063640246,"Inspection of Left Pleural Cavity, Open Approach",4.8,1.0


,INC_KEY,PRIMARYECODEICD10,AGEYEARS,SEX,EDDISCHARGEDISPOSITION,HOSPDISCHARGEDISPOSITION,MECHANISM,TRAUMATYPE
736,220063640246,"Assault by handgun discharge, initial encounter",58.0,Male,Deceased/expired,NaN,Firearm,Penetrating
783,220063640302,Pedestrian on foot injured in collision with c...,79.0,Male,Deceased/expired,NaN,MVT Pedestrian,Blunt
1302,220063657375,"Assault by unspecified firearm discharge, init...",28.0,Male,Operating Room,Deceased/Expired,Firearm,Penetrating
1754,220063698586,"Assault by unspecified firearm discharge, init...",17.0,Male,Deceased/expired,NaN,Firearm,Penetrating
3289,220063858158,Pedestrian on foot injured in collision with h...,38.0,Male,Operating Room,Deceased/Expired,MVT Pedestrian,Blunt


## Devising way to get thoracotomies that started on the left `ONLY`

> Shouldn't be any right thoracotomies before left in true EDT protocol.

> Also found in the data that there are additional thoracotomies performed on the same patient, sometimes as short as within 30 minutes of each other (i.e. at 18 min and 48 min following arrival to the ED)

> Below code block needs to be tuned up to work correctly

# What questions should I be asking next?

1. Age guideline/ cutoff
    - Create age vs mortality graph, hue = injury type
    - Can compare over the years
    - Histogram of age
2. Mortality based on timing and performing EDT (mortality rate too excessive if not performed within x minutes)
    - Mortality vs. PROCEDUREMINS for thoracotomy, hue = injury type
3. Complications/ comorbidity
    - probably has to be in a table format
4. Evaluate WTA/ EAST guidelines vs not followed
    - will be more complicated, could mainly focus on pre-hospital cpr time and sol for criteria

# Progress:

> Was able to generate table with some information (not everything) combined, in theory I think this is enough to do the project.

> Next steps for today would be to create a dataframe that has `ED Mortality`, `Hospital Mortality` and `Overall Mortality` calculated for `2017 -  2022`. 

> We can use unique `INC_KEY` to select the unique individuals for the mortality calculations. In terms of data exploration of the data available, we can make the following figures:
    - Histogram of age
    - Bar plot comparing female vs male cases
    - Plot showing number of `ICDPROCEDURECODE` used for various mechanisms of injury
    - Plot showing people who died in the ED vs hospital
    - Plot of injury type
    - Plot of mechanism (i.e. firearm etc.)
    - Plot of time to first procedure for each of the potential thoracotomy labels
    - ED/ HOSP mortality vs age, hue = MOI, 